In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型對於電鋸資料集的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]

#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    train_loss_TL = []
    train_accuracy_TL = []
    val_loss_TL = []
    val_accuracy_TL = []
    test1_accuracy_TL = []
    test3_accuracy_TL = []
    test5_accuracy_TL = []
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    
    #資料抓入
    test_loader1 = epg.getTestDataset(sound[0])
    test_loader3 = epg.getTestDataset(sound[1])
    test_loader5 = epg.getTestDataset(sound[2])
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("TL: ", TL+1, "fold: ", fold)
        print("----------------------------------------------------------------------------------------------------------------")
        #資料抓入
        train, val = epg.getTrainDataset(fold, sound[TL], num_class)
        #分批方法
        train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
        val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
        
        #優化器
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        #調度器
        scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        #訓練模型
        train_loss, train_accuracy, val_loss, val_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[TL], num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5)
    
        #將gpu資料取回
        train_loss = torch.stack(train_loss,dim=-1).cpu()
        train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
        val_loss = torch.stack(val_loss,dim=-1).cpu()
        val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
        for i in range(num_epochs):
            # 寫入資料
            f.write("TL :{} fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(TL+1 ,fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        train_loss_TL.append(train_loss)
        train_accuracy_TL.append(train_accuracy)
        val_loss_TL.append(val_loss)
        val_accuracy_TL.append(val_accuracy)
        test1_accuracy_TL.append(test1_accuracy)
        test3_accuracy_TL.append(test3_accuracy)
        test5_accuracy_TL.append(test5_accuracy)
    modelName = "Fold_" + str(fold) + "_TF.pkl"
    torch.save(model, modelName)
    train_loss_array.append(train_loss_TL)
    train_accuracy_array.append(train_accuracy_TL)
    val_loss_array.append(val_loss_TL)
    val_accuracy_array.append(val_accuracy_TL)
    test1_accuracy_array.append(test1_accuracy_TL)
    test3_accuracy_array.append(test3_accuracy_TL)
    test5_accuracy_array.append(test5_accuracy_TL)
f.close()

----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.6979429721832275 Train_acc: 37.125000% Val_Loss: 1.5296066999435425 Val_accuracy: 55.250000%
Test accuracy of the model: 66.800000%
Test accuracy of the model: 72.000000%
Test accuracy of the model: 83.000000%
Train Epoch: 2/30 Train_Loss: 1.19143807888031 Train_acc: 60.312500% Val_Loss: 1.486228585243225 Val_accuracy: 56.750000%
Test accuracy of the model: 65.200000%
Test accuracy of the model: 65.333333%
Test accuracy of the model: 73.000000%
Train Epoch: 3/30 Train_Loss: 1.010512351989746 Train_acc: 65.625000% Val_Loss: 1.4651803970336914 Val_accuracy: 60.000000%
Test accuracy of the model: 84.000000%
Test accuracy of the model: 93.666667%
Test accuracy of the model: 96.000000%
Train Epoch: 4/30 Train_Loss: 1.0510

Train Epoch: 3/30 Train_Loss: 0.20426982641220093 Train_acc: 94.895836% Val_Loss: 0.94671231508255 Val_accuracy: 80.000008%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 4/30 Train_Loss: 0.2356618493795395 Train_acc: 95.625008% Val_Loss: 0.6443008780479431 Val_accuracy: 75.833336%
Test accuracy of the model: 96.800000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 5/30 Train_Loss: 0.13359491527080536 Train_acc: 94.791672% Val_Loss: 1.081362247467041 Val_accuracy: 77.083336%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 6/30 Train_Loss: 0.18218094110488892 Train_acc: 95.416672% Val_Loss: 0.6494347453117371 Val_accuracy: 81.250008%
Test accuracy of the model: 96.600000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 7/30 Train_Loss

Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 6/30 Train_Loss: 0.055241603404283524 Train_acc: 100.000000% Val_Loss: 0.35510551929473877 Val_accuracy: 81.250000%
Test accuracy of the model: 96.800000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 7/30 Train_Loss: 0.06803925335407257 Train_acc: 99.687500% Val_Loss: 0.8368359804153442 Val_accuracy: 81.250000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 8/30 Train_Loss: 0.04979395493865013 Train_acc: 99.375000% Val_Loss: 0.7545214891433716 Val_accuracy: 86.250000%
Test accuracy of the model: 96.800000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 9/30 Train_Loss: 0.03209835663437843 Train_acc: 100.000000% Val_Loss: 0.7051478028297424 Val_accuracy: 83.750000%
Test accuracy of the model: 96.600000%
Test accuracy of the m

Test accuracy of the model: 95.000000%
Test accuracy of the model: 98.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 9/30 Train_Loss: 0.6442239284515381 Train_acc: 80.000000% Val_Loss: 0.8401974439620972 Val_accuracy: 77.000000%
Test accuracy of the model: 95.200000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 10/30 Train_Loss: 0.5230922102928162 Train_acc: 80.250000% Val_Loss: 0.7394366264343262 Val_accuracy: 74.500000%
Test accuracy of the model: 93.600000%
Test accuracy of the model: 97.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 11/30 Train_Loss: 0.4737173020839691 Train_acc: 82.500000% Val_Loss: 0.9883331060409546 Val_accuracy: 79.250000%
Test accuracy of the model: 96.000000%
Test accuracy of the model: 98.333333%
Test accuracy of the model: 99.000000%
Train Epoch: 12/30 Train_Loss: 0.5768392086029053 Train_acc: 83.812500% Val_Loss: 0.5332351922988892 Val_accuracy: 81.750000%
Test accuracy of the model: 9

Test accuracy of the model: 96.200000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 12/30 Train_Loss: 0.12913310527801514 Train_acc: 97.500008% Val_Loss: 0.33415570855140686 Val_accuracy: 90.833336%
Test accuracy of the model: 96.400000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 13/30 Train_Loss: 0.0812339037656784 Train_acc: 97.708336% Val_Loss: 0.32813918590545654 Val_accuracy: 90.416672%
Test accuracy of the model: 96.400000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 14/30 Train_Loss: 0.16871649026870728 Train_acc: 97.395836% Val_Loss: 0.3924963176250458 Val_accuracy: 91.666672%
Test accuracy of the model: 96.400000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 15/30 Train_Loss: 0.14573417603969574 Train_acc: 96.875008% Val_Loss: 0.2031290978193283 Val_accuracy: 92.500008%
Test accuracy of th

...

Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 25/30 Train_Loss: 0.35824692249298096 Train_acc: 89.125000% Val_Loss: 0.6755043268203735 Val_accuracy: 82.500000%
Test accuracy of the model: 96.600000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 26/30 Train_Loss: 0.3561844825744629 Train_acc: 91.125000% Val_Loss: 0.43725454807281494 Val_accuracy: 82.750000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 27/30 Train_Loss: 0.30619823932647705 Train_acc: 90.750000% Val_Loss: 0.40519776940345764 Val_accuracy: 82.500000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 28/30 Train_Loss: 0.22263449430465698 Train_acc: 90.625000% Val_Loss: 0.6905243992805481 Val_accuracy: 82.250000%
Test accuracy of the model: 97.000000%
Test accuracy of th

Test accuracy of the model: 97.200000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 28/30 Train_Loss: 0.14544370770454407 Train_acc: 98.125008% Val_Loss: 0.4023412764072418 Val_accuracy: 92.500008%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
Train Epoch: 29/30 Train_Loss: 0.09571675211191177 Train_acc: 98.125008% Val_Loss: 0.7712700963020325 Val_accuracy: 91.250008%
Test accuracy of the model: 97.200000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 30/30 Train_Loss: 0.139293372631073 Train_acc: 98.229172% Val_Loss: 0.19531665742397308 Val_accuracy: 90.416672%
Test accuracy of the model: 96.800000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
----------------------------------------------------------------------------------------------------------------
TL:  3 fold:  4
--------------------

Train Epoch: 30/30 Train_Loss: 0.022638997063040733 Train_acc: 100.000000% Val_Loss: 0.745940089225769 Val_accuracy: 93.750000%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 100.000000%
----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  5
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.595354437828064 Train_acc: 38.500000% Val_Loss: 1.7481684684753418 Val_accuracy: 45.000000%
Test accuracy of the model: 85.400000%
Test accuracy of the model: 94.666667%
Test accuracy of the model: 96.000000%
Train Epoch: 2/30 Train_Loss: 1.4647722244262695 Train_acc: 53.250000% Val_Loss: 1.4345656633377075 Val_accuracy: 61.250000%
Test accuracy of the model: 78.600000%
Test accuracy of the model: 84.333333%
Test accuracy of the model: 88.000000%
Train Epoch: 3/30 Train_Loss:

Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 2/30 Train_Loss: 0.4148724675178528 Train_acc: 93.020836% Val_Loss: 0.6107947826385498 Val_accuracy: 82.083336%
Test accuracy of the model: 96.600000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 3/30 Train_Loss: 0.2007531225681305 Train_acc: 93.854172% Val_Loss: 0.7698860764503479 Val_accuracy: 80.833336%
Test accuracy of the model: 97.000000%
Test accuracy of the model: 99.666667%
Test accuracy of the model: 100.000000%
Train Epoch: 4/30 Train_Loss: 0.24292398989200592 Train_acc: 94.375008% Val_Loss: 0.3696185052394867 Val_accuracy: 82.916672%
Test accuracy of the model: 96.400000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 5/30 Train_Loss: 0.15534517168998718 Train_acc: 94.270836% Val_Loss: 0.4992940127849579 Val_accuracy: 86.250008%
Test accuracy of the model: 96.800000%
Test accuracy of the model:

Test accuracy of the model: 95.600000%
Test accuracy of the model: 99.000000%
Test accuracy of the model: 99.000000%
Train Epoch: 5/30 Train_Loss: 0.0514795258641243 Train_acc: 100.000000% Val_Loss: 0.4189318120479584 Val_accuracy: 85.000000%
Test accuracy of the model: 96.400000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 6/30 Train_Loss: 0.05064508691430092 Train_acc: 99.375000% Val_Loss: 0.8432127237319946 Val_accuracy: 87.500000%
Test accuracy of the model: 96.800000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 7/30 Train_Loss: 0.06704851239919662 Train_acc: 99.687500% Val_Loss: 1.035332441329956 Val_accuracy: 83.750000%
Test accuracy of the model: 96.600000%
Test accuracy of the model: 99.333333%
Test accuracy of the model: 100.000000%
Train Epoch: 8/30 Train_Loss: 0.054040588438510895 Train_acc: 100.000000% Val_Loss: 0.5029327273368835 Val_accuracy: 85.000000%
Test accuracy of the mod

In [4]:
#設定存檔位置
f = open("lastData.txt", "w")
for TL in range(3):
    train_loss_last = []
    train_accuracy_last = []
    val_loss_last = []
    val_accuracy_last = []
    test1_accuracy_last = []
    test3_accuracy_last = []
    test5_accuracy_last = []
    
    #整理出訓練完的數值
    for fold in range(5):
        train_loss_last.append(train_loss_array[fold][TL][num_epochs-1])
        train_accuracy_last.append(train_accuracy_array[fold][TL][num_epochs-1])
        val_loss_last.append(val_loss_array[fold][TL][num_epochs-1])
        val_accuracy_last.append(val_accuracy_array[fold][TL][num_epochs-1])
        test1_accuracy_last.append(test1_accuracy_array[fold][TL][num_epochs-1])
        test3_accuracy_last.append(test3_accuracy_array[fold][TL][num_epochs-1])
        test5_accuracy_last.append(test5_accuracy_array[fold][TL][num_epochs-1])
        # 寫入資料
        f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][TL][num_epochs-1], train_accuracy_array[fold][TL][num_epochs-1], val_loss_array[fold][TL][num_epochs-1], val_accuracy_array[fold][TL][num_epochs-1], test1_accuracy_array[fold][TL][num_epochs-1], test3_accuracy_array[fold][TL][num_epochs-1], test5_accuracy_array[fold][TL][num_epochs-1]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    # 寫入資料
    f.write("Max Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(val_loss_last), max(val_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
    f.write("Min Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(val_loss_last), min(val_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
    f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(val_loss_last), np.mean(val_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
f.close()